#Movie reviews - VADER

In [ ]:
##raw unlabeled text- VADER-

In [ ]:
import nltk

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#takes in a string and return a dictionary of scores into 4 categories -neg,positive,neutral,compout

In [ ]:
sid=SentimentIntensityAnalyzer()

In [ ]:
a="This is a good movie"

In [ ]:

sid.polarity_scores(a)

{'compound': 0.4404, 'neg': 0.0, 'neu': 0.508, 'pos': 0.492}

In [ ]:
a = 'This was the worst film to ever disgrace the screen.'
sid.polarity_scores(a)

{'compound': -0.8074, 'neg': 0.477, 'neu': 0.523, 'pos': 0.0}

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/New folder/Colab Notebooks/NLP/UPDATED_NLP_COURSE/TextFiles/moviereviews2.tsv', sep='\t')

In [ ]:
df.head()

,label,review
0,pos,I loved this movie and will watch it again. Or...
1,pos,"A warm, touching movie that has a fantasy-like..."
2,pos,I was not expecting the powerful filmmaking ex...
3,neg,"This so-called ""documentary"" tries to tell tha..."
4,pos,This show has been my escape from reality for ...


In [ ]:
df['label'].value_counts()

pos    3000
neg    3000
Name: label, dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
blanks=[]
for i,lb,rv in df.itertuples():
  if(type(rv)==str):
    if rv.isspace():
      blanks.append(i)

In [ ]:
blanks

[]

In [ ]:
sid.polarity_scores(df.iloc[0]['review'])

{'compound': 0.872, 'neg': 0.062, 'neu': 0.695, 'pos': 0.243}

In [ ]:
df['scores']=df['review'].apply(lambda review:sid.polarity_scores(review))

In [ ]:
df

,label,review,scores
0,pos,I loved this movie and will watch it again. Or...,"{'neg': 0.062, 'neu': 0.695, 'pos': 0.243, 'co..."
1,pos,"A warm, touching movie that has a fantasy-like...","{'neg': 0.033, 'neu': 0.783, 'pos': 0.184, 'co..."
2,pos,I was not expecting the powerful filmmaking ex...,"{'neg': 0.097, 'neu': 0.795, 'pos': 0.108, 'co..."
3,neg,"This so-called ""documentary"" tries to tell tha...","{'neg': 0.116, 'neu': 0.832, 'pos': 0.052, 'co..."
4,pos,This show has been my escape from reality for ...,"{'neg': 0.031, 'neu': 0.796, 'pos': 0.173, 'co..."
...,...,...,...
5995,pos,"Of the three remakes of this plot, I like them...","{'neg': 0.028, 'neu': 0.825, 'pos': 0.148, 'co..."
5996,neg,Poor Whoopi Goldberg. Imagine her at a friend'...,"{'neg': 0.181, 'neu': 0.776, 'pos': 0.043, 'co..."
5997,neg,"Honestly before I watched this movie, I had he...","{'neg': 0.057, 'neu': 0.78, 'pos': 0.163, 'com..."
5998,pos,This movie is essentially shot on a hand held ...,"{'neg': 0.073, 'neu': 0.862, 'pos': 0.065, 'co..."


In [ ]:
df['compund']=df['scores'].apply(lambda d:d['compound'])

In [ ]:
df.head()

,label,review,scores,compund
0,pos,I loved this movie and will watch it again. Or...,"{'neg': 0.062, 'neu': 0.695, 'pos': 0.243, 'co...",0.8720
1,pos,"A warm, touching movie that has a fantasy-like...","{'neg': 0.033, 'neu': 0.783, 'pos': 0.184, 'co...",0.9549
2,pos,I was not expecting the powerful filmmaking ex...,"{'neg': 0.097, 'neu': 0.795, 'pos': 0.108, 'co...",0.7201
3,neg,"This so-called ""documentary"" tries to tell tha...","{'neg': 0.116, 'neu': 0.832, 'pos': 0.052, 'co...",-0.9821
4,pos,This show has been my escape from reality for ...,"{'neg': 0.031, 'neu': 0.796, 'pos': 0.173, 'co...",0.9874


In [ ]:
df['comp_score']=df['compund'].apply(lambda score:'pos' if score>=0 else 'neg')

In [ ]:
df

,label,review,scores,compund,comp_score
0,pos,I loved this movie and will watch it again. Or...,"{'neg': 0.062, 'neu': 0.695, 'pos': 0.243, 'co...",0.8720,pos
1,pos,"A warm, touching movie that has a fantasy-like...","{'neg': 0.033, 'neu': 0.783, 'pos': 0.184, 'co...",0.9549,pos
2,pos,I was not expecting the powerful filmmaking ex...,"{'neg': 0.097, 'neu': 0.795, 'pos': 0.108, 'co...",0.7201,pos
3,neg,"This so-called ""documentary"" tries to tell tha...","{'neg': 0.116, 'neu': 0.832, 'pos': 0.052, 'co...",-0.9821,neg
4,pos,This show has been my escape from reality for ...,"{'neg': 0.031, 'neu': 0.796, 'pos': 0.173, 'co...",0.9874,pos
...,...,...,...,...,...
5995,pos,"Of the three remakes of this plot, I like them...","{'neg': 0.028, 'neu': 0.825, 'pos': 0.148, 'co...",0.9377,pos
5996,neg,Poor Whoopi Goldberg. Imagine her at a friend'...,"{'neg': 0.181, 'neu': 0.776, 'pos': 0.043, 'co...",-0.9578,neg
5997,neg,"Honestly before I watched this movie, I had he...","{'neg': 0.057, 'neu': 0.78, 'pos': 0.163, 'com...",0.9856,pos
5998,pos,This movie is essentially shot on a hand held ...,"{'neg': 0.073, 'neu': 0.862, 'pos': 0.065, 'co...",-0.1010,neg


In [ ]:
accuracy_score(df['label'],df['comp_score'])

0.7319397993311036

In [ ]:
print(classification_report(df['label'],df['comp_score']))

              precision    recall  f1-score   support

         neg       0.83      0.58      0.68      2990
         pos       0.68      0.88      0.77      2990

    accuracy                           0.73      5980
   macro avg       0.76      0.73      0.73      5980
weighted avg       0.76      0.73      0.73      5980



In [ ]:
print(confusion_matrix(df['label'],df['comp_score']))

[[1734 1256]
 [ 347 2643]]


#Movie reviews - tfidf vectorizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/New folder/Colab Notebooks/NLP/UPDATED_NLP_COURSE/TextFiles/moviereviews2.tsv', sep='\t')

In [ ]:
df.head()

,label,review
0,pos,I loved this movie and will watch it again. Or...
1,pos,"A warm, touching movie that has a fantasy-like..."
2,pos,I was not expecting the powerful filmmaking ex...
3,neg,"This so-called ""documentary"" tries to tell tha..."
4,pos,This show has been my escape from reality for ...


###  Check for missing values:

In [ ]:
# Check for NaN values:
df.isnull().sum()

label      0
review    20
dtype: int64

In [ ]:
# Check for whitespace strings (it's OK if there aren't any!):


blanks=[]

for i,lb,rv in df.itertuples():
  if type(rv)==str:
    if(rv.isspace()):
      blanks.append(i)


print(blanks)






[]


###Remove NaN values:

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

label     0
review    0
dtype: int64

### Take a quick look at the `label` column:

In [ ]:
X=df['review']
y=df['label']

###  Split the data into train & test sets:


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=42)




### Build a pipeline to vectorize the date, then train and fit a model


In [ ]:
text_clf=Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])



### Run predictions and analyze the results

In [ ]:
# Form a prediction set
text_clf.fit(X_train,y_train)
pre=text_clf.predict(X_test)


In [ ]:
# Report the confusion matrix
print(confusion_matrix(y_test,pre))


[[900  91]
 [ 63 920]]


In [ ]:
# Print a classification report
print(classification_report(y_test,pre))

              precision    recall  f1-score   support

         neg       0.93      0.91      0.92       991
         pos       0.91      0.94      0.92       983

    accuracy                           0.92      1974
   macro avg       0.92      0.92      0.92      1974
weighted avg       0.92      0.92      0.92      1974



In [ ]:
# Print the overall accuracy
print(accuracy_score(y_test,pre))


0.9219858156028369
